In [1]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
# Paths and site names setup
waves_folder_path = "./dataset_Ondas"
shorelines_folder_path = "./dataset_linhascosta"
transects_folder_path = "./dataset_transects"
site_names = ["CCFT", "NNOR", "MEIA", "TROI"]

In [3]:
# Initialize dictionaries to store DataFrames
data, annual_data = {}, {}

In [4]:
# Processing wave and shoreline data for each site
for name in site_names:
    # File paths
    waves_file_path = os.path.join(waves_folder_path, f"{name}_wave_timeseries.csv")
    shorelines_file_path = os.path.join(shorelines_folder_path, f"{name}_shoreline_timeseries.csv")
    transects_file_path = os.path.join(transects_folder_path, f"{name}_transects.geojson")

    # Read the waves CSV files into DataFrame
    waves_df = pd.read_csv(waves_file_path, sep=',', header=0) # Set header=0 to use the first row as column headers
    waves_df['time'] = pd.to_datetime(waves_df['time'])
    waves_df.set_index('time', inplace=True)
    waves_df['years'] = waves_df.index.year
    waves_df['months'] = waves_df.index.month
    waves_df.index = pd.MultiIndex.from_tuples(
    [(year, month) for year, month in zip(waves_df.index.year, waves_df.index.month)],
    names=['years', 'months'])
    waves_df = waves_df[waves_df['years'] != 1983] # Remove 1983 because satellite data for shorelines is not available for that year
    
    # List of directions (16 directions compass rose)
    directions = ['N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE', 'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NNW']
    def degrees_to_direction(wave_direction_degrees):
        if wave_direction_degrees >= 0 and   wave_direction_degrees <= 11.25:
            return 'N'
        elif wave_direction_degrees <= 33.75:
            return 'NNE'
        elif wave_direction_degrees <= 56.25:
            return 'NE'
        elif wave_direction_degrees <= 78.75:
            return 'ENE'
        elif wave_direction_degrees <= 101.25:
            return 'E'
        elif wave_direction_degrees <= 123.75:
            return 'ESE'
        elif wave_direction_degrees <= 146.25:
            return 'SE'
        elif wave_direction_degrees <= 168.75:
            return 'SSE'
        elif wave_direction_degrees <= 191.25:
            return 'S'
        elif wave_direction_degrees <= 213.75:
            return 'SSW'
        elif wave_direction_degrees <= 236.25:
            return 'SW'
        elif wave_direction_degrees <= 258.75:
            return 'WSW'
        elif wave_direction_degrees <= 281.25:
            return 'W'
        elif wave_direction_degrees <= 303.75:
            return 'WNW'
        elif wave_direction_degrees <= 326.25:
            return 'NW'
        elif wave_direction_degrees <= 348.75:
            return 'NNW'
        elif wave_direction_degrees <= 360:
            return 'N'
        else:
            return 'false'
  
    # One-hot encode the 'mwd' column
    waves_df['mwd'] = waves_df['mwd'].apply(degrees_to_direction)

    # Create a DataFrame of dummy variables for 'mwd'
    one_hot_encode = pd.get_dummies(waves_df['mwd'], prefix='from')

    # Concatenate the one-hot encoded columns to the original DataFrame
    waves_df = pd.concat([waves_df, one_hot_encode], axis=1)
    waves_df = waves_df.drop('mwd', axis=1)

    # Iterate through directions and create new columns for each direction's pp1d and swh
    for direction in directions:
        # Create new columns for pp1d and swh
        pp1d_column_name = f'pp1d_from_{direction}'
        swh_column_name = f'swh_from_{direction}'
    
        # Use boolean indexing to set values based on the condition
        waves_df[pp1d_column_name] = waves_df['pp1d'] * waves_df[f'from_{direction}']
        waves_df[swh_column_name] = waves_df['swh'] * waves_df[f'from_{direction}']
    
    # Drop the original 'mwd' column and the 'pp1d' and 'swh' columns
    waves_df.drop(columns=[f'from_{direction}' for direction in directions], inplace=True)
    waves_df.drop(columns=['pp1d','swh'], inplace=True)
    
    
    # Read the shorelines CSV files into DataFrame
    shorelines_df = pd.read_csv(shorelines_file_path)
    shorelines_df = shorelines_df.iloc[:, 1:]
    shorelines_df['dates'] = pd.to_datetime(shorelines_df['dates'])
    shorelines_df.set_index('dates', inplace=True)
    shorelines_df['years'] = shorelines_df.index.year
    shorelines_df['months'] = shorelines_df.index.month
    shorelines_df.index = pd.MultiIndex.from_tuples(
    [(year, month) for year, month in zip(shorelines_df.index.year, shorelines_df.index.month)],
    names=['years', 'months'])
    
    # Add a new column to waves and shorelines dataframes to indicate the site name
    waves_df['site'] = name
    shorelines_df['site'] = name

    # Read the transects GeoJSON file into a GeoDataFrame
    transects_gdf = gpd.read_file(transects_file_path, driver='GeoJSON')

    # Add DataFrames to the dictionary with site name as key
    data[name] = {
        'waves': waves_df,
        'shorelines': shorelines_df,
        'transects': transects_gdf
    }
    


In [9]:
# Iterate over keys in the data dictionary
for name in data.keys():
    waves_df = data[name]['waves']
    
    # Group by 'years' and calculate quantiles for each column
    wave_df_annual = waves_df.groupby(level=['years', 'months']).agg(
           {
        'pp1d_from_N': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None),
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None),
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None)
        ],
        'swh_from_N': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None),
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None),
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None)
        ],
        'pp1d_from_NNE': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None),
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None),
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None)
        ],
        'swh_from_NNE': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None),
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None),
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None)
        ],
        'pp1d_from_NE': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None),
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None),
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None)
        ],
        'swh_from_NE': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None),
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None),
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None)
        ],
        'pp1d_from_ENE': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None),
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None),
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None)
        ],
        'swh_from_ENE': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None),
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None),
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None)
        ],
        'pp1d_from_E': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None),
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None),
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None) 
        ],
        'swh_from_E': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None), 
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None), 
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None) 
        ],
        'pp1d_from_ESE': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None), 
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None), 
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None) 
        ],
        'swh_from_ESE': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None), 
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None), 
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None) 
        ],
        'pp1d_from_SE': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None), 
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None), 
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None) 
        ], 
        'swh_from_SE': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None), 
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None), 
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None) 
        ],
        'pp1d_from_SSE': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None), 
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None), 
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None) 
        ],
        'swh_from_SSE': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None), 
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None), 
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None)
        ],
        'pp1d_from_S': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None),
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None),
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None) 
        ],
        'swh_from_S': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None), 
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None),
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None) 
        ],
        'pp1d_from_SSW': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None),
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None),
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None) 
        ],
        'swh_from_SSW': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None),
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None),
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None)
        ],
        'pp1d_from_SW': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None),
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None),
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None) 
        ],
        'swh_from_SW': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None),
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None),
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None)
        ],
        'pp1d_from_WSW': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None),
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None),
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None) 
        ],
        'swh_from_WSW': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None),
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None),
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None)
        ],
        'pp1d_from_W': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None), 
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None), 
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None)
        ],
        'swh_from_W': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None), 
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None), 
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None)
        ],
        'pp1d_from_WNW': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None), 
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None), 
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None)
        ],
        'swh_from_WNW': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None), 
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None), 
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None)
        ],
        'pp1d_from_NW': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None), 
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None), 
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None)
        ],
        'swh_from_NW': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None), 
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None), 
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None)
        ],
        'pp1d_from_NNW': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None),
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None),
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None)
        ],
        'swh_from_NNW': [
            ('10th_quantile', lambda x: x[x != 0].quantile(0.1) if any(x != 0) else None),
            ('50th_quantile', lambda x: x[x != 0].quantile(0.5) if any(x != 0) else None),
            ('90th_quantile', lambda x: x[x != 0].quantile(0.9) if any(x != 0) else None)
        ]})
    

    # Replace NaN values with zero
    wave_df_annual = wave_df_annual.fillna(0)

    shoreline_df = data[name]['shorelines']

    # Group by 'years' and calculate median for each column
    shoreline_df_annual = shoreline_df.groupby(level=['years', 'months']).median(numeric_only=True)
    
    # Drop year and month columns
    shoreline_df_annual = shoreline_df_annual.drop(['years', 'months'], axis=1)

    # Iterate over each column in the DataFrame

    for i in range(1, len(shoreline_df_annual.columns) - 1):
        col = shoreline_df_annual.columns[i]
        prev_col = shoreline_df_annual.columns[i - 1] if i - 1 >= 0 else None
        next_col = shoreline_df_annual.columns[i + 1] if i + 1 < len(shoreline_df_annual.columns) else None

        # Check if there are any NaN values in the current column
        if shoreline_df_annual[col].isnull().any():
            # Fill NaN values with the mean of the available previous and next columns
            if prev_col is not None and next_col is not None:
                shoreline_df_annual[col] = (shoreline_df_annual[prev_col] + shoreline_df_annual[next_col]) / 2
            elif prev_col is not None:
                shoreline_df_annual[col] = shoreline_df_annual[prev_col]
            elif next_col is not None:
                shoreline_df_annual[col] = shoreline_df_annual[next_col]
            else:
                # If there are no immediate previous and next columns, extend the search to 3 columns
                prev_cols = [shoreline_df_annual.columns[j] for j in range(i - 2, i) if j >= 0]
                next_cols = [shoreline_df_annual.columns[j] for j in range(i + 1, i + 4) if j < len(shoreline_df_annual.columns)]

                available_cols = prev_cols + next_cols

                # Filter out None values (columns that are out of range)
                available_cols = [col for col in available_cols if col is not None]

                # Take the mean of available columns
                if len(available_cols) > 0:
                    shoreline_df_annual[col] = shoreline_df_annual[available_cols].mean(axis=1)
                    
        
        for column in shoreline_df_annual.columns:
        # Check if there are any NaN values in the column
        if shoreline_df_annual[column].isnull().any():
            
            # Final check and fill/drop remaining NaNs
            shoreline_df_annual.fillna(method='ffill', inplace=True)
            shoreline_df_annual.fillna(method='bfill', inplace=True)
            
            # Calculate the median value of the column (excluding NaN values)
            median_value = shoreline_df_annual[column].median()
        
            # Replace NaN values with the calculated median value
            shoreline_df_annual[column].fillna(median_value, inplace=True)

        # Ensure no NaNs are left before model training
        if shoreline_df_annual.isna().any().any():
            print(f"NaNs remain in shorelines data for {name}")
            continue  # Skip this iteration if NaNs are still present

    
    
    
    
    # Add the DataFrame to the dictionary with site name as key
    annual_data[name] = {
        'waves': wave_df_annual,
        'shorelines': shoreline_df_annual
    }

In [10]:
annual_data['TROI']['waves']

pp1d_from_N                                swh_from_N  \
             10th_quantile 50th_quantile 90th_quantile 10th_quantile   
years months                                                           
1984  1            0.00000      0.000000      0.000000      0.000000   
      2           11.00150     11.651595     12.257721      0.581329   
      3           10.03587     10.035870     10.035870      0.782094   
      4            0.00000      0.000000      0.000000      0.000000   
      5            0.00000      0.000000      0.000000      0.000000   
...                    ...           ...           ...           ...   
2022  8            0.00000      0.000000      0.000000      0.000000   
      9            0.00000      0.000000      0.000000      0.000000   
      10           0.00000      0.000000      0.000000      0.000000   
      11           0.00000      0.000000      0.000000      0.000000   
      12           0.00000      0.000000      0.000000      0.000000   

                                         pp1d_from_NNE                \
             50th_quantile 90th_quantile 10th_quantile 50th_quantile   
years months                                                           
1984  1           0.000000      0.000000      0.000000      0.000000   
      2           0.698613      0.849621     11.091750     12.218543   
      3           0.782094      0.782094     10.085448     10.130714   
      4           0.000000      0.000000      0.000000      0.000000   
      5           0.000000      0.000000      0.000000      0.000000   
...                    ...           ...           ...           ...   
2022  8           0.000000      0.000000      0.000000      0.000000   
      9           0.000000      0.000000      0.000000      0.000000   
      10          0.000000      0.000000      0.000000      0.000000   
      11          0.000000      0.000000      0.000000      0.000000   
      12          0.000000      0.000000      0.000000      0.000000   

                            swh_from_NNE  ...  pp1d_from_NW   swh_from_NW  \
             90th_quantile 10th_quantile  ... 90th_quantile 10th_quantile   
years months                              ...                               
1984  1           0.000000      0.000000  ...     16.568388      1.159329   
      2          12.235737      0.602527  ...     15.556421      0.944696   
      3          10.176769      0.842339  ...     16.012798      0.664541   
      4           0.000000      0.000000  ...     11.969103      0.695217   
      5           0.000000      0.000000  ...     11.222614      0.719590   
...                    ...           ...  ...           ...           ...   
2022  8           0.000000      0.000000  ...     11.344798      0.656596   
      9           0.000000      0.000000  ...     16.501251      1.084615   
      10          0.000000      0.000000  ...     12.812888      0.663309   
      11          0.000000      0.000000  ...     14.569991      1.552865   
      12          0.000000      0.000000  ...      0.000000      0.000000   

                                         pp1d_from_NNW                \
             50th_quantile 90th_quantile 10th_quantile 50th_quantile   
years months                                                           
1984  1           2.113522      3.003425     10.858882     11.489670   
      2           1.942769      2.496285      8.990899     10.270372   
      3           1.757484      2.532742     10.067748     10.614656   
      4           1.110330      1.613585      0.000000      0.000000   
      5           1.001971      1.661999      7.460350      7.915628   
...                    ...           ...           ...           ...   
2022  8           0.987281      1.394260      8.006328      8.218338   
      9           1.306514      1.547684      8.186714      8.565018   
      10          1.016424      1.373330      6.105875      6.203791   
      11          1.781020      2.520800      0.000000

In [ ]:
# Combine standardized data into a single empty DataFrame with all months and years from the previous tables
combined_shorelines = pd.DataFrame(index=pd.MultiIndex.from_product(
    [shoreline_df.index.get_level_values(0).unique(), shoreline_df.index.get_level_values(1).unique()],
    names=['years', 'months']))

In [ ]:
# Merge data from all sites - SHORELINES
for name in site_names:
    df_to_merge = annual_data[name]['shorelines']

    # Reset index if 'years' and 'months' are part of the index
    if 'years' in df_to_merge.index.names and 'months' in df_to_merge.index.names:
        df_to_merge = df_to_merge.reset_index()

    combined_shorelines = combined_shorelines.merge(df_to_merge, 
                                                   on=['years', 'months'], 
                                                   how='left')

# Handling NaNs - Group by 'years' and fill NaNs with the mean of the respective year
for column in combined_shorelines.columns:
    if column not in ['site', 'years', 'months']:
        combined_shorelines[column] = combined_shorelines.groupby('years')[column].transform(lambda x: x.fillna(x.mean()))



In [ ]:
wave_df_annual

In [ ]:
# making sure that the waves dataframe has the same dimensions as the shorelines

combined_waves = pd.DataFrame(index=pd.MultiIndex.from_product(
    [shoreline_df.index.get_level_values(0).unique(), shoreline_df.index.get_level_values(1).unique()],
    names=['years', 'months']))


In [ ]:
# Merge data from all sites - SHORELINES

df_to_merge_waves = wave_df_annual

# Reset index if 'years' and 'months' are part of the index
if 'years' in df_to_merge_waves.index and 'months' in df_to_merge.index_waves:
    df_to_merge_waves = df_to_merge_waves.reset_index()

combined_waves = combined_waves.merge(df_to_merge_waves, 
                                      on=['years', 'months'], 
                                      how='left')

In [ ]:
#combined_waves.shape
combined_shorelines.shape

In [ ]:
# Preparing the data
X = combined_waves  # Assuming 'combined_waves' contains your features
y = combined_shorelines  # Assuming 'combined_shorelines' contains your target variables

In [ ]:
X

In [ ]:
#reassigning multi-index to combined_shorelines table
#combined_shorelines = combined_shorelines.set_index(['years', 'months'])
combined_shorelines

In [ ]:
# Splitting the data into training and testing sets

# Create training datasets
x_train = combined_waves[combined_waves.index.get_level_values(0) <= 2014]
y_train = combined_shorelines[combined_shorelines.index.get_level_values(0) <= 2014]

# Create testing datasets
x_test = combined_waves[combined_waves.index.get_level_values(0) > 2014]
y_test = combined_shorelines[combined_shorelines.index.get_level_values(0) > 2014]

In [ ]:
y_train

In [ ]:
# Create an instance of the DecisionTreeRegressor model
model = DecisionTreeRegressor()


In [ ]:
# Train the model
model.fit(x_train, y_train)


In [ ]:
# Use the model to predict shoreline positions
y_pred = model.predict(x_test)
y_pred = pd.DataFrame(y_pred, columns=y_test.columns, index=y_test.index)
y_pred

In [ ]:
# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse